In [40]:
import numpy as np

In [41]:
# set lookback value 
gap = 10
def combination(data):
    dataset = []
    for idx,value in enumerate(data):
        if(idx<len(data)-gap):
            temp = data[idx:idx+gap]
            temp = temp.reshape(6*gap)
            dataset.append(temp)
    return np.array(dataset)

def get_data():#normlize
    data = np.loadtxt('hk.txt')
    print(data.shape)
    x = combination(data)
    y = data[gap:]
    return x,y

x,y = get_data()

(2280, 6)


In [42]:
def get_test():
    data = np.loadtxt('lotto.txt').astype('int8')
    print(data.shape)
    x = combination(data)
    y = data[gap:]
    return x,y 

test_x,test_y = get_test()

(1470, 6)


In [43]:
# Decision trees
from sklearn import tree
tree_model = tree.DecisionTreeRegressor().fit(x,y)
tree_result = tree_model.predict(test_x).astype('int8')

In [44]:
#Generalized Linear Models
from sklearn import linear_model
linear_model = linear_model.LinearRegression().fit(x,y)
linear_result = np.around(linear_model.predict(test_x)).astype('int8')

In [ ]:
# Deep NN with keras
from keras.models import Sequential 
from keras.layers.core import Dense, Dropout
def get_DNN():
    model = Sequential()
    model.add(Dense(512, input_dim=6*gap,activation='tanh'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='tanh'))
    model.add(Dropout(0.4))
    model.add(Dense(6,activation='sigmoid'))# 01~49中任選6個號碼
    model.summary()
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

dnn_model = get_DNN()
dnn_model.fit(x/50, y/50, batch_size=128, epochs=1,verbose=1)
dnn_result =np.around(dnn_model.predict(test_x/50)*50).astype('int8')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               31232     
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 6)                 1542      
Total params: 164,102
Trainable params: 164,102
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
2270/2270 [==============================] - 1s 306us/step - loss: 0.0380


In [ ]:
# LSTM  with keras
from keras.layers import Embedding,LSTM
def get_LSTM():
    model = Sequential()
    model.add(Embedding(6*gap,32))
    model.add(LSTM(512,recurrent_dropout=0.2, dropout=0.2))
    model.add(Dense(256, activation="tanh"))
    model.add(Dense(6, activation="sigmoid")) 
    model.summary()
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

lstm_model = get_LSTM()
lstm_model.fit(x/50, y/50, batch_size=128, epochs=1,verbose=1)
lstm_result =np.around(lstm_model.predict(test_x/50)*50).astype('int8')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          1920      
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               1116160   
_________________________________________________________________
dense_19 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 1542      
Total params: 1,250,950
Trainable params: 1,250,950
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
1792/2270 [======================>.......] - ETA: 12s - loss: 0.0453

In [ ]:
print(tree_result.shape)
print(linear_result.shape)
print(dnn_result.shape)
print(lstm_result.shape)

In [ ]:
def test():
    countRd = 0
    countTree = 0
    countLinear = 0
    countDNN = 0
    countLSTM = 0

    winRd = 0
    winTree = 0
    winLinear = 0
    winDNN = 0
    winLSTM = 0

    for idx in range(len(test_y)):
        print("------開獎對比----"+str(idx))
        rand = np.sort((np.add(np.random.choice(49,6, replace=False),1)))
        print(rand)
        print(linear_result[idx])
        print(tree_result[idx])
        print(dnn_result[idx])
        print(lstm_result[idx])
        print(test_y[idx])

        print("-------------------")
        setRd = set(rand)
        setTree = set(linear_result[idx])
        setLinear = set(tree_result[idx]) 
        setDNN = set(dnn_result[idx])
        setLSTM = set(lstm_result[idx])
        setWin = set(test_y[idx])

        resultRd = setRd&setWin
        resultTree = setTree&setWin
        resultLinear = setLinear&setWin
        resultDNN = setDNN&setWin
        resultLSTM = setLSTM&setWin

        if(resultRd!=set()):
            print('Random:',resultRd)
            if(len(resultRd)>=3):
                print('Random 中獎咯')
                winRd += 1
            countRd += 1
        if(resultTree!=set()):
            print('Decision Tree:',resultTree)
            if(len(resultTree)>=3):
                print('Decision Tree 中獎咯')
                winTree +=1
            countTree += 1
        if(resultLinear!=set()):
            print('Linear:',resultLinear)
            if(len(resultLinear)>=3):
                print('Linear 中獎咯')
                winLinear +=1
            countLinear += 1
        if(resultDNN!=set()):
            print('DNN:',resultDNN)
            if(len(resultDNN)>=3):
                print('DNN 中獎咯')
                winDNN +=1
            countDNN += 1
        if(resultLSTM!=set()):
            print('LSTM:',resultLSTM)
            if(len(resultLSTM)>=3):
                print('LSTM 中獎咯')
                winLSTM +=1
            countLSTM += 1

    print("-------------------")
    print('hit','win')
    print(countRd,winRd)
    print(countTree,winTree)
    print(countLinear,winLinear)
    print(countDNN,winDNN)
    print(countLSTM,winLSTM)
    count = []
    win =[]
    count.append([countRd,countTree,countLinear,countDNN,countLSTM])
    win.append([winRd,winTree,winLinear,winDNN,winLSTM])
    return count,win

In [ ]:
count = []
win = []
for i in range(20):
    c,w = test()
    count.append(c)
    win.append(w)

In [ ]:
count = np.array(count).reshape(20,5)
win = np.array(win).reshape(20,5)

In [ ]:
print(count[:,0].mean())
print(win[:,0].mean())

In [ ]:
gap